In [ ]:
!pip install request flask wiotp-sdk paho-mqtt cloudant datetime

In [ ]:
import wiotp.sdk.application
import json
from cloudant.client import Cloudant
from datetime import datetime

serviceUsername = "CloudanrUsername-bluemix"                                      ## Cloudant Username
servicePassword = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX" ## Cloudant Password
serviceURL = "https://CloudanrUsername-bluemix.cloudantnosqldb.appdomain.cloud"
clients = Cloudant(serviceUsername, servicePassword, url=serviceURL)
clients.connect();

myConfig = { 
    "auth" :{
        "key": "a-ORG-KEY",
        "token": "PASSWORD"
    }
}

my_database = clients['c4c']

def myEventCallback(event):
    try:
        print(event.eventId)
        if(event.eventId=="message"):
            now = datetime.now()
            current_time = now.strftime("%H:%M:%S")
            my_database = clients['c4c']
            myData={
                "time" : current_time,
                "device": event.data["device"],
                "lat" : event.data["lat"],
                "lng" : event.data["lng"],
                "data" : event.data["data"]
                }    
            my_document = my_database.create_document(myData)
            if my_document.exists():
                print('SUCCESS!!')
            my_database = clients['c4c']
            file="{'data':["
            for document in my_database:
                file+=str(document)
                file+=","
            file=file[:-1]
            file+="]}"
            file = file.replace("'",'"')
            file=json.loads(file)
            client.publishEvent("webpage", "webpage", "update", "json", file)
            print("Update")
        elif (event.eventId=="init"):
            my_database = clients['c4c']
            file="{'data':["
            for document in my_database:
                file+=str(document)
                file+=","
            file=file[:-1]
            file+="]}"
            file = file.replace("'",'"')
            file=json.loads(file)
            client.publishEvent("webpage", "webpage", "update", "json", file)
            print("Update")
            
    except KeyError as error:
        ...
    
    
try:
    
    client = wiotp.sdk.application.ApplicationClient(config=myConfig)
    client.connect()
    client.deviceEventCallback = myEventCallback
    client.subscribeToDeviceEvents()

    while 1:
        ...
except KeyboardInterrupt:
    client.disconnect()